In [1]:
%load_ext autoreload
%load_ext autotime

time: 405 µs (started: 2023-03-04 13:31:54 -08:00)


In [2]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 1.06 s (started: 2023-03-04 13:31:54 -08:00)


In [3]:
import dask.dataframe as dd
import numpy as np
import pandas as pd


time: 694 ms (started: 2023-03-04 13:31:55 -08:00)


In [4]:
%autoreload 2

from wisio.recorder import RecorderAnalyzer

log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-cosmoflow/nodes-32/_parquet"

recorder_analyzer = RecorderAnalyzer(
    cluster_manager_args=dict(
#         force_local=True,
        cluster_settings=dict(
#             cores=4,
            dashboard_port=3696,
            local_directory="/var/tmp/dask-recorder/cosmoflow",
            log_file="%J.log",
            worker_queue="pdebug"
        )
    ),
    working_dir='.recorder',
    debug=True
) 

recorder_analyzer

time: 827 ms (started: 2023-03-04 13:31:55 -08:00)


In [5]:
[client.dashboard_link for client in recorder_analyzer.cluster_manager.clients]

['http://192.168.66.99:3696/status']

time: 3.76 ms (started: 2023-03-04 13:31:56 -08:00)


In [6]:
%autoreload 2

res = recorder_analyzer.analyze_parquet(log_dir=log_dir)

KeyboardInterrupt: 

time: 8min 57s (started: 2023-03-04 13:32:16 -08:00)


In [ ]:
views, bottlenecks = res

In [ ]:
views[('trange',)]['bottlenecks'].head()

tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOLoop object at 0x2000321aff60>>, <Task finished coro=<SpecCluster._correct_state_internal() done, defined at /usr/WS2/iopp/projects/vani-analysis-tool/virenv-lassen/lib/python3.7/site-packages/distributed/deploy/spec.py:310> exception=OSError(12, 'Cannot allocate memory')>)
Traceback (most recent call last):
  File "/usr/WS2/iopp/projects/vani-analysis-tool/virenv-lassen/lib/python3.7/site-packages/tornado/ioloop.py", line 740, in _run_callback
    ret = callback()
  File "/usr/WS2/iopp/projects/vani-analysis-tool/virenv-lassen/lib/python3.7/site-packages/tornado/ioloop.py", line 764, in _discard_future_result
    future.result()
  File "/usr/WS2/iopp/projects/vani-analysis-tool/virenv-lassen/lib/python3.7/site-packages/distributed/deploy/spec.py", line 402, in _close
    await self._correct_state()
  File "/usr/WS2/iopp/projects/vani-ana

In [15]:
'file_id' in views[('trange',)]['expanded_view'].columns

True

time: 2.58 ms (started: 2023-03-02 08:13:51 -08:00)


In [16]:
views[('file_id',)]['bottleneck_view'].head()

,trange,file_id,proc_id,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,...,sequential_time,random_time,sequential_size,random_size,sequential_count,random_count,duration_score,duration_cut,duration_pero,duration_perr
1283,4,4531734245058167199,6860131432589748884,3,0,0.000003,1,0,0,0,...,0.000000,0,0,0,0,0,5,0.5,7.047217e-07,2.987094e-08
1284,5,4531734245058167199,6860131432589748884,3,0,1.970968,1,0,0,0,...,0.000000,0,0,0,0,0,5,0.5,4.789599e-01,2.030161e-02
1285,6,4531734245058167199,6860131432589748884,5,0,0.000172,6,5530,5530,5530,...,0.000103,0,5530,0,1,0,5,0.5,4.182159e-05,1.772686e-06
1286,9,4531734245058167199,6860131432589748884,5,0,0.000873,7,5530,5530,5530,...,0.000100,0,5530,0,1,0,5,0.5,2.121455e-04,8.992182e-06
1287,11,4531734245058167199,6860131432589748884,5,0,0.000873,7,5530,5530,5530,...,0.000095,0,5530,0,1,0,5,0.5,2.120240e-04,8.987032e-06


time: 206 ms (started: 2023-03-02 08:16:26 -08:00)


In [17]:
from wisio._recorder.analysis import compute_unique_file_names, compute_unique_proc_names
%autoreload 2

unique_file_names = compute_unique_file_names(log_dir=log_dir)
unique_proc_names = compute_unique_proc_names(log_dir=log_dir)


time: 66.2 ms (started: 2023-03-02 08:16:27 -08:00)


In [27]:
from wisio._recorder.bottlenecks import RecorderBottleneckDetector

bottleneck_detector = RecorderBottleneckDetector(
    log_dir=log_dir,
    views=views,
    view_types=['trange', 'file_id', 'proc_id'],
    unique_file_names=unique_file_names,
    unique_proc_names=unique_proc_names
)

bottlenecks = bottleneck_detector.detect_bottlenecks(max_io_time=max_io_time)

len(bottlenecks)


15

time: 25.9 s (started: 2023-03-02 08:41:13 -08:00)


In [28]:
bottlenecks[('proc_id', 'trange')]

{5: {'llc': {'duration_sum': 2.1918428,
   'index_count': 32,
   'size_min': 0,
   'size_max': 0,
   'size_sum': 0,
   'func_id': ['open'],
   'read_time': 0.0,
   'write_time': 0.0,
   'metadata_time': 2.191842794418335,
   'read_size': 0,
   'write_size': 0,
   'metadata_size': 0,
   'read_count': 0,
   'write_count': 0,
   'metadata_count': 32,
   'data_count': 0,
   'data_size': 0,
   'data_time': 0.0,
   'sequential_time': 0.0,
   'random_time': 0,
   'sequential_size': 0,
   'random_size': 0,
   'sequential_count': 0,
   'random_count': 0,
   'duration_pero': 0.53263414,
   'duration_perr': 0.022585722,
   'duration_score': 5,
   'duration_cut': 0.5},
  'proc_id': {'0': {'llc': {'duration_sum': 1.970968,
     'index_count': 1,
     'size_min': 0,
     'size_max': 0,
     'size_sum': 0,
     'func_id': ['open'],
     'read_time': 0.0,
     'write_time': 0.0,
     'metadata_time': 1.9709680080413818,
     'read_size': 0,
     'write_size': 0,
     'metadata_size': 0,
     'read_cou

time: 73.9 ms (started: 2023-03-02 08:41:41 -08:00)


In [29]:
bottlenecks[('proc_id', 'trange', 'file_id')]

{'/p/gpfs1/iopp/temp/cm1r20.3.2846827/config_files/les_ConvBoundLayer/cm1out_stats.ctl': {'llc': {'duration_sum': 2.2200522,
   'index_count': 2528,
   'size_min': 0,
   'size_max': 5530,
   'size_sum': 1996330,
   'func_id': ['__fxstat',
    '__xstat',
    'close',
    'ftruncate',
    'lseek',
    'open',
    'write'],
   'read_time': 0.0,
   'write_time': 0.12472909987263847,
   'metadata_time': 2.095323208289983,
   'read_size': 0,
   'write_size': 1996330,
   'metadata_size': 0,
   'read_count': 0,
   'write_count': 361,
   'metadata_count': 2167,
   'data_count': 361,
   'data_size': 1996330,
   'data_time': 0.12472909987263847,
   'sequential_time': 0.12472909987263847,
   'random_time': 0,
   'sequential_size': 1996330,
   'random_size': 0,
   'sequential_count': 361,
   'random_count': 0,
   'duration_pero': 0.53948927,
   'duration_perr': 0.022877984,
   'duration_score': 1815,
   'duration_cut': 181.5},
  'proc_id': {'0': {'llc': {'duration_sum': 2.2200522,
     'index_count

time: 601 ms (started: 2023-03-02 08:44:30 -08:00)


In [24]:
views[('trange',)]['bottleneck_view'] \
    .groupby(['file_id', 'proc_id', 'trange']) \
    .first() \
    .drop(columns=['acc_pat', 'io_cat'], errors='ignore') \
    .merge(unique_file_names, left_index=True, right_index=True) \
    .merge(unique_proc_names, left_index=True, right_index=True) \
    .compute()

duration_sum  index_count  \
file_id             proc_id             trange                              
4531734245058167199 6860131432589748884 5           1.970968            1   
4531734247156679037 6860131432592370324 5           1.939239            1   
                    6860131432594991764 5           2.167557            1   
                    6860131432597613204 5           2.129455            1   
                    6860131432600234644 5           2.069059            1   
                    6860131432602856084 5           2.151693            1   
                    6860131432605477524 5           2.054112            1   
                    6860131432608098964 5           2.139112            1   
                    6860131432610720404 5           2.054134            1   
                    6860131432613341844 5           2.165936            1   
                    6860131432615963284 5           2.191390            1   
                    6860131432618584724 5           1.992225            1   
                    6860131432621206164 5           2.114179            1   
                    6860131432623827604 5           2.164664            1   
                    6860131432626449044 5           2.164769            1   
                    6860131432629070484 5           2.191843            1   
                    6860131432631691924 5           2.099467            1   
                    6860131432634313364 5           1.970162            1   
                    6860131432636934804 5           2.164277            1   
                    6860131432639556244 5           1.952886            1   
                    6860131432642177684 5           1.994138            1   
                    6860131432644799124 5           2.054732            1   
                    6860131432647420564 5           2.164203            1   
                    6860131432650042004 5           2.010935            1   
                    6860131432652663444 5           2.164720            1   
                    6860131432655284884 5           2.151723            1   
                    6860131432657906324 5           1.952563            1   
                    6860131432660527764 5           2.116145            1   
                    6860131432663149204 5           2.116290            1   
                    6860131432665770644 5           2.026798            1   
                    6860131432668392084 5           2.025340            1   
                    6860131432671013524 5           2.151049            1   

                                                size_min  size_max  size_sum  \
file_id             proc_id             trange                                 
4531734245058167199 6860131432589748884 5              0         0         0   
4531734247156679037 6860131432592370324 5              0         0         0   
                    6860131432594991764 5              0         0         0   
                    6860131432597613204 5              0         0         0   
                    6860131432600234644 5              0         0         0   
                    6860131432602856084 5              0         0         0   
                    6860131432605477524 5              0         0         0   
                    6860131432608098964 5              0         0         0   
                    6860131432610720404 5              0         0         0   
                    6860131432613341844 5              0         0         0   
                    6860131432615963284 5              0         0         0   
                    6860131432618584724 5              0         0         0   
                    6860131432621206164 5              0         0         0   
                    6860131432623827604 5              0         0         0   
                    6860131432626449044 5              0         0         0   
                    6860131432629070484 5              0         0   

time: 381 ms (started: 2023-03-01 21:10:02 -08:00)
